In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import datetime
import geopandas as gpd
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)
faults = pd.read_csv('../data/J1939Faults.csv', low_memory = False )
diagnostics = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv', low_memory = False)
service_fault_code = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx')

/Applications/anaconda/anaconda3/envs/geospatial_ds6/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
faults

In [ ]:
faults.dtypes 

In [27]:
diagnostics

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1
...,...,...,...,...
12821621,12864020,EngineCoolantTemperature,181.4,1248457
12821622,12864021,ParkingBrake,False,1248457
12821623,12864022,SwitchedBatteryVoltage,14.1,1248457
12821624,12864023,DistanceLtd,28606.65625,1248457


In [31]:
diagnostics['Name'].unique()

array(['IgnStatus', 'EngineOilPressure', 'EngineOilTemperature',
       'TurboBoostPressure', 'EngineLoad', 'AcceleratorPedal',
       'IntakeManifoldTemperature', 'FuelRate', 'FuelLtd', 'EngineRpm',
       'LampStatus', 'BarometricPressure', 'FuelLevel', 'Speed',
       'EngineTimeLtd', 'CruiseControlSetSpeed', 'CruiseControlActive',
       'EngineCoolantTemperature', 'ParkingBrake',
       'SwitchedBatteryVoltage', 'DistanceLtd', 'Throttle',
       'FuelTemperature', 'ServiceDistance'], dtype=object)

In [32]:
diagnostics['Name'].nunique()

24

In [28]:
service_fault_code 

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,Y,9996,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7120,Y,9997,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7121,Y,9998,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7122,Y,9999,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN


#### 36.0666667, -86.4347222 = 193 Aldi Blvd, Mount Juliet, TN  37122, United States
#### 35.5883333, -86.4438888 = 119–121 Eady Rd, Shelbyville, TN  37160, United States
#### 36.1950, -83.174722 = 5754 Fish Hatchery Rd, Russellville, TN  37860, United States

In [3]:
# Define the service locations
service_locations = [(36.0666667, -86.4347222), (35.5883333, -86.4438888), (36.1950, -83.174722)]

# Filter out faults occurring within 1 mile of service locations
for loc in service_locations:
    lat_mile = 2/69 # Convert 1 mile to degrees of latitude
    lon_mile =  1/(69 * np.cos(np.radians(loc[0]))) * 2 # Convert 2 mile to degrees of longitude at the given latitude
    faults = faults[(np.abs(faults['Latitude'] - loc[0]) > lat_mile) | (np.abs(faults['Longitude'] - loc[1]) > lon_mile)]
    
# Remove rows with Equipment ID > 5
faults = faults[faults['EquipmentID'].str.len() <= 5]
faults

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000
1187331,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1187332,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1187333,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [4]:
# pivot the diagnostics dataframe
wide_diag = diagnostics.pivot(index='FaultId', columns='Name', values='Value')
wide_diag

Name,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultId,,,,,,,,,,,,,,,,,,,,,,,,
1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1248455,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,1310.25,10722.7,96.4,58979.184415546,7.647805,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1248456,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,1340.75,9326.75,100,65080.10587046,8.995086,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96


In [5]:
# Merge fault and diagnostics dataframe
fault_diag = pd.merge(left = faults,
         right = wide_diag,
         left_on = 'RecordID',
         right_on = 'FaultId'
        )
fault_diag

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051207,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1051208,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,1310.25,10722.7,96.4,58979.184415546,7.647805,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1051209,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,1340.75,9326.75,100,65080.10587046,8.995086,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
1051210,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000,1.6,14.4275,False,67.72946,28606.65625,181.4,0,27.26,221.7312,863.25,586.75,23.6,4042.49282573,0,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [6]:
fault_diag['EquipmentID'].nunique()

1043

In [7]:
fault_diag['EquipmentID'].unique()

array(['1439', '1369', '1674', ..., '2356', '2380', '2381'], dtype=object)

In [8]:
fault_diag['spn'].unique()

array([   111,    629,   1807,   4364,     97,   1067,     96,    829,
          596,    929,  50353,   1569,    102,   1068,    611,    791,
          907,    793,   5444,   5396,    100,   2863,    790,    789,
          101,    639,    411,     91,   3226,    905,   3821,   2791,
          627,    614, 523531,   5848,   3936,    792,      0,    157,
         3610,   3216,   3480,    800,   1761,   5743,   5851,   3251,
         3556,    103,    797,   1045, 523530,   5319,   3509,    810,
         1209,   3058,   5246,   3217,   4334,   3362,    248,   3513,
         1059,    630,   1231,    641,    563,   4794,    168,    811,
           81,   5397,    803,   3482,   2659,   3064,    188,    105,
         3363,   5491,   3364,   4363,   4094,   1808,   3720, 523543,
         3360,   4346,   4376,   3361,   4340,   4342,   3031,   4344,
         3242,   4096,   3060,    862,   1483,    917,    184,    245,
         1028,   3697,   3698,   3703,   5298,  65535,  51923,   4607,
      

In [9]:
fault_diag['spn'].nunique()

420

In [10]:
fault_diag[fault_diag['spn'] == 5246]['EquipmentID'].nunique() 
# There are 191 equipment with spn/fault code 5246

191

In [12]:
# creating list of unique EquipmentID with full derate. 
equipment = fault_diag[fault_diag['spn'] == 5246]['EquipmentID'].unique()

In [13]:
equipment = np.sort(equipment,kind='quicksort')
equipment

array(['1329', '1339', '1366', '1373', '1375', '1378', '1383', '1384',
       '1389', '1391', '1395', '1396', '1399', '1401', '1403', '1407',
       '1417', '1418', '1419', '1431', '1437', '1440', '1443', '1444',
       '1452', '1453', '1458', '1472', '1473', '1477', '1486', '1487',
       '1488', '1490', '1492', '1501', '1508', '1509', '1519', '1521',
       '1524', '1525', '1528', '1529', '1534', '1535', '1537', '1539',
       '1543', '1547', '1549', '1551', '1552', '1554', '1556', '1557',
       '1559', '1560', '1561', '1563', '1564', '1566', '1567', '1571',
       '1572', '1573', '1575', '1579', '1581', '1582', '1584', '1585',
       '1586', '1590', '1591', '1592', '1595', '1598', '1599', '1600',
       '1601', '1602', '1603', '1604', '1605', '1623', '1630', '1637',
       '1643', '1654', '1657', '1659', '1661', '1663', '1665', '1668',
       '1669', '1683', '1686', '1689', '1691', '1692', '1698', '1704',
       '1711', '1718', '1739', '1742', '1743', '1745', '1746', '1748',
      

In [14]:
# filter rows based on values present in an equipment array
# filtering only for equipment that undergone full derate
fault_diag_filtered = fault_diag[fault_diag['EquipmentID'].isin(equipment)]

In [15]:
fault_diag_filtered

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
5,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000,48,14.4275,False,64.6226,470381.4,181.4,30,38.28,196.5313,1514.5,9480,44,70349.809963756,4.583399,NaN,True,111.2,1023,NaN,NaN,13.6022,3276.75,NaN,6.67
9,10,990446,2015-02-21 11:41:55.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1417,105438630,33.039953,-96.182592,2015-02-21 11:41:51.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
10,11,990459,2015-02-21 11:42:19.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1582,105410980,40.158055,-80.151018,2015-02-21 11:42:55.000,NaN,14.1375,False,64.6226,257885.9,181.4,80,34.22,218.975,1382.375,5085.35,42.8,37724.957799834,13.72378,NaN,True,78.8,1023,NaN,NaN,64.82649,3276.75,NaN,17.69
13,14,990470,2015-02-21 11:43:40.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1582,105410980,40.161944,-80.161018,2015-02-21 11:43:36.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
15,16,990495,2015-02-21 11:45:06.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00190002*082113134117*07700053*I0*BBZ*,79623056,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1601,105430420,38.272037,-78.937129,2015-02-21 11:45:02.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051183,1248429,123886552,2020-03-06 12:11:54.000,High Voltage (Left Fuel Level Sensor),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,False,126,NaN,1853,105376821,26.079814,-80.123518,2020-03-06 12:11:49.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65535,NaN,NaN,NaN,NaN,NaN,NaN
1051185,1248431,123891846,2020-03-06 12:20:36.000,High Voltage (Fuel Level),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,True,126,NaN,1853,105376821,26.078518,-80.122268,2020-03-06 12:41:17.000,0,14.645,False,66.48672,443434.9,186.8,12,22.62,203.7875,601,10755.85,54,62063.92146675,0.7132668,32,True,143.6,1279,True,NaN,0,NaN,100,0.58
1051186,1248432,123891847,2020-03-06 12:20:36.000,High Voltage (Left Fuel Level Sensor),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,True,126,NaN,1853,105376821,26.078518,-80.122268,2020-03-06 12:41:17.000,0,14.645,False,66.48672,443434.9,186.8,12,22.62,203.7875,601,10755.85,54,62063.92146675,0.7132668,32,True,143.6,1279,True,NaN,0,NaN,100,0.58
1051192,1248438,123893761,2020-03-06 12:51:53.000,High Voltage (Fuel Level),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,False,126,NaN,1853,105376821,26.078888,-80.122361,2020-03-06 12:51:49.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65535,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
fault_diag_filtered[fault_diag_filtered['spn'] == 5246]['EquipmentID'].nunique()

191

In [17]:
fault_diag_filtered['spn'].unique()

array([   111,    929,   1068,    611,    791,    907,   5444,   5396,
          596,    790,    789,   2863,    905,   3821,   4364,     91,
         5848,   3936,    792,    157,  50353,    101,   3480,    800,
         5743,   5851,     97,   3216,    103,     96,    829,   3226,
         1569,   1045, 523530,    627,    810,   5246,   3217,   4334,
         3362,    630,   1059,   1761,    614,   5319, 523531,     81,
         5397,   3610,    188,   3251,   5491,   3364,   3482,   3058,
          102,   3360,   4346,   4376,   3361,   3363,   4340,   4342,
         3031,   4344,   3242,    639,   1231,    629,    862,    168,
          794,    803,    723,    412,   1481,    641,    184,    190,
         1078,   3720,   5113,   3556,    612,   1483,   1815,    649,
         2791,   1668,   1236,    632,   1072,    563,   4276,   1209,
         2795,   4375,    248,     94,  51923,   1807,    795,   3464,
         1808,   4794,    624,     84,   4096,    110,    655,    245,
      

In [18]:
fault_diag_filtered['spn'].nunique()
# 317 different fault codes/ spn associated with equiments undergone full derate

318

In [ ]:
# fault_diag.loc[fault_diag['spn'].isin([5246, 1569])]

In [19]:
# Counts of different faults 
fault_diag_filtered['spn'].value_counts()

111     164856
929      34037
96       27672
829      25589
1068      5899
         ...  
38           1
4382         1
4380         1
904          1
1073         1
Name: spn, Length: 318, dtype: int64

In [20]:
fault_diag_filtered['spn'].value_counts().loc[lambda x : x > 50] 

111     164856
929      34037
96       27672
829      25589
1068      5899
         ...  
653         56
5298        55
652         52
3363        51
5443        51
Name: spn, Length: 139, dtype: int64

In [21]:
fault_diag_filtered.groupby(['EquipmentID', 'spn'])['spn'].count()

EquipmentID  spn 
1329         111     28
             1569     2
             1761     1
             2863    12
             3031     2
                     ..
306          3226     4
             4094     1
             5246     1
             5742     1
             5743     1
Name: spn, Length: 4926, dtype: int64

In [22]:
fault_diag_filtered.groupby('EquipmentID')['spn'].unique()

EquipmentID
1329    [4344, 5246, 1569, 3031, 3242, 2863, 50353, 11...
1339    [596, 4364, 627, 50353, 1045, 51923, 723, 5394...
1366    [4364, 596, 50353, 4794, 111, 3216, 1569, 5246...
1373    [111, 563, 4363, 4364, 100, 91, 639, 2791, 102...
1375    [3480, 4364, 1045, 111, 3216, 5397, 5319, 3251...
                              ...                        
301     [639, 596, 3226, 5396, 632, 190, 1761, 641, 38...
302     [5444, 5396, 190, 1078, 1761, 96, 829, 1045, 1...
304     [3217, 1059, 596, 614, 829, 96, 248, 624, 101,...
305     [611, 791, 907, 96, 829, 596, 1761, 3226, 641,...
306     [596, 627, 639, 248, 1028, 96, 829, 84, 1231, ...
Name: spn, Length: 191, dtype: object

In [23]:
fault_diag_filtered.groupby(['EquipmentID'])['spn'].count()

EquipmentID
1329      67
1339      78
1366     426
1373    1226
1375    1409
        ... 
301      784
302     3119
304     2207
305     4950
306      265
Name: spn, Length: 191, dtype: int64

In [24]:
fault_diag_filtered[fault_diag_filtered['spn'] == 5246]['EquipmentID'].value_counts().loc[lambda x : x > 1] 

1524    60
1535    44
305     28
1539    25
1525    25
        ..
1395     2
1582     2
1599     2
1508     2
2009     2
Name: EquipmentID, Length: 174, dtype: int64

In [25]:
fault_diag_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336147 entries, 5 to 1051193
Data columns (total 44 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   RecordID                   336147 non-null  int64  
 1   ESS_Id                     336147 non-null  int64  
 2   EventTimeStamp             336147 non-null  object 
 3   eventDescription           323619 non-null  object 
 4   actionDescription          0 non-null       float64
 5   ecuSoftwareVersion         277286 non-null  object 
 6   ecuSerialNumber            269330 non-null  object 
 7   ecuModel                   326452 non-null  object 
 8   ecuMake                    326452 non-null  object 
 9   ecuSource                  336147 non-null  int64  
 10  spn                        336147 non-null  int64  
 11  fmi                        336147 non-null  int64  
 12  active                     336147 non-null  bool   
 13  activeTransitionCount      3

In [26]:
fault_diag_filtered[(fault_diag_filtered['EquipmentID'] == '1524') & (fault_diag_filtered['spn'] == 5246)]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
300071,336266,6698415,2016-01-06 14:07:24.000,NaN,NaN,NaN,NaN,NaN,NaN,0,5246,16,True,1,NaN,1524,105450466,35.873379,-87.859537,2016-01-06 14:08:01.000,NaN,14.7175,NaN,NaN,412663.1,183.2,100,45.24,213.0125,1313,8502.75,90.4,59974.375,9.536641,183.2,True,104,1279,NaN,NaN,63,13.75,NaN,19.72
300468,336704,6705137,2016-01-06 19:46:33.000,NaN,NaN,NaN,NaN,NaN,NaN,0,5246,16,False,1,NaN,1524,105450466,35.031481,-89.891018,2016-01-06 19:46:29.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
300469,336705,6705138,2016-01-06 19:45:54.000,NaN,NaN,NaN,NaN,NaN,NaN,0,5246,0,True,1,NaN,1524,105450466,35.031481,-89.891018,2016-01-06 19:46:30.000,NaN,14.79,NaN,NaN,NaN,145.4,22,42.92,NaN,599.25,8506.3,78.4,59991.75,1.083109,NaN,True,122,1279,NaN,NaN,0,13.35,NaN,NaN
300471,336707,6705192,2016-01-06 19:51:20.000,NaN,NaN,NaN,NaN,NaN,NaN,0,5246,0,False,1,NaN,1524,105450466,35.031435,-89.891018,2016-01-06 19:51:15.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
300498,336739,6705868,2016-01-06 20:23:45.000,NaN,NaN,NaN,NaN,NaN,NaN,0,5246,0,True,2,NaN,1524,105450466,35.031435,-89.891111,2016-01-06 20:56:21.000,NaN,14.79,NaN,NaN,NaN,136.4,0,0,133.1938,0,8506.35,78.4,59991.875,0,136.4,True,129.2,1279,NaN,NaN,0,12.75,NaN,0
301661,338023,6728189,2016-01-07 20:59:42.000,NaN,NaN,NaN,NaN,NaN,NaN,0,5246,0,False,1,NaN,1524,105450466,35.059259,-89.997685,2016-01-07 20:20:22.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
311838,350158,6922965,2016-01-18 15:57:16.000,NaN,NaN,NaN,NaN,NaN,NaN,0,5246,15,True,1,NaN,1524,105450466,36.121574,-85.777685,2016-01-18 15:56:16.000,NaN,14.5725,NaN,NaN,415091,181.4,93,42.92,213.0688,1326.75,8555.7,50,60353.375,15.79754,181.4,True,89.6,1279,NaN,NaN,63.5,13.85,NaN,28.42
311892,350226,6924016,2016-01-18 16:57:53.000,NaN,NaN,NaN,NaN,NaN,NaN,0,5246,15,False,1,NaN,1524,105450466,35.986342,-85.013148,2016-01-18 16:57:49.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
311893,350228,6924018,2016-01-18 16:57:14.000,NaN,NaN,NaN,NaN,NaN,NaN,0,5246,16,True,1,NaN,1524,105450466,35.986342,-85.013148,2016-01-18 16:57:50.000,NaN,13.9925,NaN,NaN,415140.4,212,20,37.12,217.5688,1095,8556.7,43.2,60360.875,4.041845,212,True,161.6,1279,NaN,NaN,0,13.95,NaN,4.93
311940,350286,6924853,2016-01-18 17:42:58.000,NaN,NaN,NaN,NaN,NaN,NaN,0,5246,16,False,1,NaN,1524,105450466,35.986296,-85.013148,2016-01-18 17:42:54.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# split location time stamp column
# convert the column to datetime format
fault_diag_filtered['EventTimeStamp'] = pd.to_datetime(fault_diag_filtered['EventTimeStamp'])

# extract the date and time components into separate columns
fault_diag_filtered['date'] = fault_diag_filtered['EventTimeStamp'].dt.date
fault_diag_filtered['time'] = fault_diag_filtered['EventTimeStamp'].dt.time

# print the updated DataFrame
fault_diag_filtered


In [ ]:
fault_diag_filtered[(fault_diag_filtered['EquipmentID'] == '1524') & (fault_diag_filtered['spn'] == 5246)].groupby('date').count()

In [ ]:

# get the current time
current_time = datetime.datetime.now().time()

# set the time component of the datetime column to the current time
df['date_time'] = df['date_time'].apply(lambda x: datetime.datetime.combine(x.date(), current_time))

# print the updated DataFrame
print(df)
